In [22]:
import os, sys
os.path.dirname(sys.executable)

'C:\\Users\\IvanB\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\tuw-nlp2024-HQEixHnN-py3.12\\Scripts'

In [23]:
# Define the new path
new_path = r'C:\Users\IvanB\OneDrive - TU Wien\University\TU Wien\2024W\194.093 Natural Language Processing and Information Extraction\Ex\TUW-NLP2024'
os.chdir(new_path)
# Verify the change
print(os.getcwd())

C:\Users\IvanB\OneDrive - TU Wien\University\TU Wien\2024W\194.093 Natural Language Processing and Information Extraction\Ex\TUW-NLP2024


In [24]:
from tuwnlp.utils import Language, TokenType, LabelLevel
from tuwnlp.utils import read_file_as_string, read_files_to_df
from tuwnlp.utils import get_low_lvl_label_mappings, get_top_lvl_label_mappings
from tuwnlp.utils import get_file_labels_dataframe

from pathlib import Path
from enum import Enum
import pandas as pd
from pandas import DataFrame
import numpy as np

In [25]:
# Define the new path
new_path = r'C:\Users\IvanB\OneDrive - TU Wien\University\TU Wien\2024W\194.093 Natural Language Processing and Information Extraction\Ex\TUW-NLP2024\notebooks'
os.chdir(new_path)
# Verify the change
print(os.getcwd())

C:\Users\IvanB\OneDrive - TU Wien\University\TU Wien\2024W\194.093 Natural Language Processing and Information Extraction\Ex\TUW-NLP2024\notebooks


In [26]:
en_naratives_labels = get_file_labels_dataframe(
    Path("../data"),
    Language.EN,
    LabelLevel.NARATIVES
)

en_texts = read_files_to_df(Path("../data/tmp"), Language.EN)
en_texts.index = en_texts["file name"].values
en_texts = en_texts.drop(columns = ["file name", "file path"])
df = pd.merge(en_texts, en_naratives_labels, left_index=True, right_index=True)


200it [00:00, 324.89it/s]


### Bag of words model

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


#Split into features and target
text_col = "text"
y_cols = [col for col in df.columns if col != text_col]
X = df[text_col]
y = df[y_cols]

#build bag of words
embedding_size = 1024
vectorizer = CountVectorizer(max_features=embedding_size)
X = vectorizer.fit_transform(X)

#build model
model = MultiOutputClassifier(MultinomialNB())
model.fit(X,y)

#evaluate model
y_pred = model.predict(X)
macro = f1_score(y, y_pred, average="macro")
micro = f1_score(y, y_pred, average="micro")
weighted = f1_score(y, y_pred, average="weighted")
macro, micro, weighted

C:\Users\IvanB\AppData\Local\pypoetry\Cache\virtualenvs\tuw-nlp2024-HQEixHnN-py3.12\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\IvanB\AppData\Local\pypoetry\Cache\virtualenvs\tuw-nlp2024-HQEixHnN-py3.12\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.6464538043169837, 0.6893424036281179, 0.7158793543104183)

### Define models

In [31]:
print(X.dtypes)
print(X.head())

object
EN_CC_100000    1\tpentagon\tPROPN\tpentagon 2\tplans\tNOUN\tp...
EN_CC_100002    1\toxford\tPROPN\toxford 2\tresidents\tPROPN\t...
EN_CC_100003    1\tfonda\tPROPN\tfonda 2\theads\tVERB\thead 3\...
EN_CC_100004    1\ttesla\tPROPN\ttesla 2\towner\tNOUN\towner 3...
EN_CC_100005    1\tclimate\tNOUN\tclimate 2\tcrazies\tNOUN\tcr...
Name: text, dtype: object


In [28]:
from itertools import product
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
from tqdm import tqdm
# LANGUAGES = [Language.BG, Language.EN, Language.HI, Language.PT]
LANGUAGES = [Language.EN]
LEVELS = [LabelLevel.NARATIVES, LabelLevel.SUBNARATIVES]
TEXT_COL = "text"
RANDOM_STATE = 42

models = [
    MultiOutputClassifier(MultinomialNB()),
    #MultiOutputClassifier(SVC()),
    MultiOutputClassifier(RandomForestClassifier())
]

coombinations = product(LANGUAGES, LEVELS, models)

all_res = []

for language, level, model in tqdm(coombinations):
    labels = get_file_labels_dataframe(
    Path("../data"),
    language,
    level,
)
    # Read and split into train and test subsets
    text = read_files_to_df(Path("../data/tmp"), language)
    text.index = text["file name"].values
    text = text.drop(columns = ["file name", "file path"])
    df = pd.merge(text, labels, left_index=True, right_index=True)
    X = df[TEXT_COL]
    y = df.drop(columns=[TEXT_COL])
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE)
    
    vectorizer = CountVectorizer(max_features=embedding_size)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    res = {}
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    res["language"] = language.value
    res["level"] = level.value
    res["model"] = clone(model)
    res["macro"] = f1_score(y_test, y_pred, average="macro")
    res["micro"] = f1_score(y_test, y_pred, average="micro")
    res["weighted"] = f1_score(y_test, y_pred, average="weighted")

    all_res.append(res)

0it [00:00, ?it/s]
200it [00:00, 3153.05it/s]
C:\Users\IvanB\AppData\Local\pypoetry\Cache\virtualenvs\tuw-nlp2024-HQEixHnN-py3.12\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\IvanB\AppData\Local\pypoetry\Cache\virtualenvs\tuw-nlp2024-HQEixHnN-py3.12\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
1it [00:00,  3.80it/s]
200it [00:00, 4028.41it/s]
C:\Users\IvanB\AppData\Local\pypoetry\Cache\virtualenvs\tuw-nlp2024-HQEixHnN-py3.12\Lib\site-packages\sklearn\metrics\_classificatio

In [29]:
df = pd.DataFrame(all_res)
df.query("level == 'NARATIVES'")

,language,level,model,macro,micro,weighted
0,EN,NARATIVES,MultiOutputClassifier(estimator=MultinomialNB()),0.241246,0.444444,0.492275
1,EN,NARATIVES,MultiOutputClassifier(estimator=RandomForestCl...,0.042311,0.365591,0.287614


### Qualitative Analysis

In [97]:
# Inverse transform the test features to get back the original text
X_test_text = vectorizer.inverse_transform(X_test)

# Loop through and print misclassified examples
misclassified_samples = []
for i in range(X_test.shape[0]):  # Use X_test.shape[0] to get the number of samples
    # Check if any label in the row is misclassified
    if (y_test.iloc[i] != y_pred[i]).any():
        # Get the misclassified example and append it to the list
        misclassified_samples.append({
            "text": " ".join(X_test_text[i]),  # Reconstruct the text from the sparse vector
            "true_labels": y_test.iloc[i].to_dict(),  # Convert true labels to dict
            "pred_labels": y_pred[i].tolist()  # Convert predicted labels to list
        })

#### sample0

In [98]:
sample0 = misclassified_samples[0]  # Accessing sample 0
# print(f"Text: {sample0['text']}\n")
# print(f"True Labels: {sample0['true_labels']}\n")
# print(f"Predicted Labels: {sample0['pred_labels']}\n")
sample0

{'text': '10 100 101 102 103 104 105 106 107 108 109 11 110 111 112 113 114 115 116 117 118 119 12 120 121 122 123 124 125 126 127 128 129 13 130 131 132 133 134 135 136 137 138 139 14 140 141 142 143 144 145 146 147 148 149 15 150 151 152 153 154 155 156 157 158 159 16 160 161 162 163 164 165 166 167 168 169 17 170 171 172 173 174 175 176 177 178 179 18 180 181 182 183 184 185 186 187 188 189 19 190 191 192 193 194 195 196 197 198 199 20 200 201 202 203 204 205 206 207 208 209 21 210 211 212 213 214 215 216 217 218 219 22 220 221 222 223 224 225 226 227 228 229 23 230 231 232 233 234 235 236 237 238 239 24 240 241 242 243 244 245 246 247 248 249 25 250 251 252 253 254 255 256 257 258 259 26 260 261 262 263 264 265 266 267 268 269 27 270 271 272 273 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 accord according act act

In [99]:
# Compare true labels with predicted labels and count mismatches
# For sample0
mismatches0 = 0
for idx, (label, true_value) in enumerate(sample0['true_labels'].items()):
    predicted_value = sample0['pred_labels'][idx]
    if true_value != predicted_value:
        mismatches0 += 1
        print(f'Mismatch found for label: "{label}"')
        print(f'True label: {true_value}')
        print(f'Predicted label: {predicted_value}\n')
print(f'Number of mismatches for sample0: {mismatches0}\n')

Mismatch found for label: "URW: Blaming the war on others rather than the invader: The West are the aggressors"
True label: True
Predicted label: False

Number of mismatches for sample0: 1



#### sample1

In [100]:
sample1 = misclassified_samples[1]  # Accessing second sample 1
# print(f"Text: {sample1['text']}\n")
# print(f"True Labels: {sample1['true_labels']}\n")
# print(f"Predicted Labels: {sample1['pred_labels']}\n")
#sample1

In [101]:
# Compare true labels with predicted labels and count mismatches
# For sample1
mismatches1 = 0
for idx, (label, true_value) in enumerate(sample1['true_labels'].items()):
    predicted_value = sample1['pred_labels'][idx]
    if true_value != predicted_value:
        mismatches1 += 1
        print(f'Mismatch found for label: "{label}"')
        print(f'True label: {true_value}')
        print(f'Predicted label: {predicted_value}\n')
print(f'Number of mismatches for sample1: {mismatches1}\n')

Mismatch found for label: "CC: Criticism of climate movement: Ad hominem attacks on key activists"
True label: False
Predicted label: True

Mismatch found for label: "CC: Downplaying climate change: Climate cycles are natural"
True label: False
Predicted label: True

Mismatch found for label: "CC: Downplaying climate change: Ice is not melting"
True label: False
Predicted label: True

Number of mismatches for sample1: 3



#### sample2

In [102]:
sample2 = misclassified_samples[2]  # Accessing second sample 1
# print(f"Text: {sample2['text']}\n")
# print(f"True Labels: {sample2['true_labels']}\n")
# print(f"Predicted Labels: {sample2['pred_labels']}\n")
#sample2

In [103]:
# Compare true labels with predicted labels and count mismatches
# For sample2
mismatches2 = 0
for idx, (label, true_value) in enumerate(sample2['true_labels'].items()):
    predicted_value = sample2['pred_labels'][idx]
    if true_value != predicted_value:
        mismatches2 += 1
        print(f'Mismatch found for label: "{label}"')
        print(f'True label: {true_value}')
        print(f'Predicted label: {predicted_value}\n')
print(f'Number of mismatches for sample2: {mismatches2}\n')

Mismatch found for label: "Other"
True label: True
Predicted label: False

Number of mismatches for sample2: 1



#### sample3

In [104]:
sample3 = misclassified_samples[3]  # Accessing second sample 1
# print(f"Text: {sample3['text']}\n")
# print(f"True Labels: {sample3['true_labels']}\n")
# print(f"Predicted Labels: {sample3['pred_labels']}\n")
#sample3

In [105]:
# Compare true labels with predicted labels and count mismatches
# For sample3
mismatches3 = 0
for idx, (label, true_value) in enumerate(sample3['true_labels'].items()):
    predicted_value = sample3['pred_labels'][idx]
    if true_value != predicted_value:
        mismatches3 += 1
        print(f'Mismatch found for label: "{label}"')
        print(f'True label: {true_value}')
        print(f'Predicted label: {predicted_value}\n')
print(f'Number of mismatches for sample3: {mismatches3}\n')

Mismatch found for label: "CC: Criticism of climate policies: Other"
True label: True
Predicted label: False

Mismatch found for label: "CC: Criticism of institutions and authorities: Criticism of political organizations and figures"
True label: True
Predicted label: False

Number of mismatches for sample3: 2



#### sample4

In [106]:
sample4 = misclassified_samples[4]  # Accessing second sample 1
# print(f"Text: {sample4['text']}\n")
# print(f"True Labels: {sample4['true_labels']}\n")
# print(f"Predicted Labels: {sample4['pred_labels']}\n")
#sample4

In [108]:
# Compare true labels with predicted labels and count mismatches
mismatches4 = 0
for idx, (label, true_value) in enumerate(sample4['true_labels'].items()):
    predicted_value = sample4['pred_labels'][idx]
    if true_value != sample4['pred_labels'][idx]:
        mismatches4 += 1
        print(f'Mismatch found for label: "{label}"')
        print(f'True label: {true_value}')
        print(f'Predicted label: {predicted_value}\n')
print(f'Number of mismatches for sample4: {mismatches4}')

Mismatch found for label: "URW: Amplifying war-related fears: Other"
True label: True
Predicted label: False

Mismatch found for label: "URW: Amplifying war-related fears: Russia will also attack other countries"
True label: True
Predicted label: False

Mismatch found for label: "URW: Amplifying war-related fears: There is a real possibility that nuclear weapons will be employed"
True label: True
Predicted label: False

Number of mismatches for sample4: 3


### Comments on the Qualitative Analysis

In reviewing the mismatches found across the 5 samples, several key observations and potential patterns were identified.

#### Inconsistent classification for similar statements

A recurring mismatch involves statements that discuss similar topics or sentiments but are classified differently by the model. For example, in sample1 "CC: Criticism of climate movement: Ad hominem attacks on key activists"** and "CC: Downplaying climate change: Ice is not melting" are classified as True in the ground truth, yet predicted as False. These errors may point to a misclassification in cases where there are nuanced debates around climate change policies or activism. These topics might overlap with broader political and social ideologies, which makes the classification task challenging.

#### Ideological complexity
Several labels in the mismatch list involve ideological positions that can be interpreted in multiple ways depending on one’s perspective. For example, i sample4 the statement "URW: Amplifying war-related fears: The West will attack other countries" is difficult because it involves predicting a future political scenario with implications for geopolitics. The label itself is a reflection of a specific ideological viewpoint (the belief that the West might provoke a larger conflict). The model may have trouble distinguishing between valid concerns and propagandist narratives.

#### "Other" label
In sample1 and sample3, many mismatches occur for statements labeled "Other". For example, the statement "CC: Criticism of climate policies: Other" is misclassified as False in sample1. The label "Other" generally indicates broad or less easily definable categories, or at least the label that applis in case none of the other labels in the same topic (in this case "Criticism of climate policies") apply. This could also point to a dataset challenge where labels within the “Other” category are either too vague or inconsistently applied, making it harder for the model to develop a reliable classification rule for such instances.

#### Ambiguity
The model seems to struggle when faced with complex or ambiguous phrasing. For instance, in sample2, the statement "CC: Criticism of climate policies: Climate policies have negative impact on the economy" was misclassified, indicating that the model may be challenged by nuanced criticisms that require a deeper understanding of the economic implications of climate policies.

Similarly, sample4 contains multiple predictions around political and war-related fears (e.g., "The West will attack other countries" and "The real possibility of nuclear weapons"), where the model might be overlooking subtle differences in the phrasing or context in these complex political arguments.

### Conclusion

Several key takeaways from the analysed mismatches (quantitative analysis) are:

- More often than not, the model incorrectly predicts 'False' when the actual label is 'True', this seems to be a pattern across the sampled cases, the only exception being sample1.

- The model may struggle because the statements involve subjective interpretations that depend on political ideology, which may cause it to misclassify them.

- Labels such as "Other" and other generalized criticisms (e.g., "Climate policies are ineffective") seem to be frequently misclassified. The model might fail to grasp the broader context or interpret these vague categories correctly, leading to errors.

- Statements that critique complex subjects like climate change policies or specific persons (e.g., "Ad hominem attacks on key activists") involve subtle language and multi-layered arguments, which might be difficult for the model to properly interpret, causing misclassification.